In [3]:
import datetime
import pandas as pd
import numpy as np
import argparse
import sys
import string
import re
import MeCab
import mojimoji

sys.path.append('../..')
from utils.func import norm_time

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_date', type=int, default=2016)
parser.add_argument('--test_date', type=int, default=2017)
# parser.add_argument('--code', type=list, default=['1711', '1712'])
parser.add_argument('--code', type=list, default=['7203', '9984'])
# parser.add_argument('--code', type=list, default=['8301', '7203', '9501', '6758', '9984',
#                                                  '8306', '8411', '6501', '6752', '6502'])

args = parser.parse_args(args=[])

code_list = [str(x) for x in args.code]

# テキストを取り出す

In [8]:
for i, date in enumerate(range(2013, 2020)):
    tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    # tmp = tmp[tmp['Company_Relevance'] == str(100)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                    'Company_Code(TSE)', 
                    'Headline', 
                   'Company_Name_J',
                    'News_Source',
                    'Company_Relevance', 
                    'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                 (tmp['News_Source'] == 'ＮＱＮ') |
                 (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                 (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])


# 株価を取り出す

In [9]:
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = int(code)
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.set_index(['date', 'code'], drop=True)
print(df2.shape)

(4350, 1)


# 銘柄名を取り出す

In [10]:
comp_json = {}
with open('../../data/company_name.csv') as f:
    line = f.readline()
    while line:
        split = line[:-1].split(',')
        comp_json[split[0]] = split[1:]
        line = f.readline()

# 形態素解析

In [11]:
m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
                                    -u ../../mydict/mydict.dic")

# /usr/local/libexec/mecab/mecab-dict-index \
# -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
# -u mydict.dic \
# -f utf-8 \
# -t utf-8 \
# mydict.csv

In [12]:
def Morph(row):
    comp_name = comp_json[str(row.name[1])]
    text = row['Headline']
    output_words = []

    # 前処理
    text = re.sub("(\*J)",  "", text)
    text = re.sub(" |　", "", text)
    text = mojimoji.zen_to_han(text, kana=False, digit=True)
    text = re.sub(r"\\u3000",  "", repr(text)[1:-1])
    text = re.sub(r"<.+>",  "", text)
    text = re.sub(r"＜.+＞",  "", text)
    text = re.sub(r"\(\d\)",  "", text)
    text = re.sub(r"|\.|（|）|「|」|｢|｣|―|：|【|】|~|\(|\)|。|/|:|\[|\]|,|\*|◇|☆|◎|;|&",  "", text)
    text = re.sub(r"\d+",  "0", text)

#     for p in string.punctuation:
#         text = text.replace(p, ' ')
    
    # company
    for name in comp_name:
        text = text.replace(name, '<company>')
    
    soup = m.parse(text)
    for row in soup.split('\n'):
        category = row.split('\t')
        if category[0] == "EOS":
            break
        else:
            
            word = category[0]            
            tag = category[1].split(',')
            if tag[2] == '人名':
                word = '<person>'
            elif tag[2] == '地域':
                word = '<location>'
            elif tag[2] == '組織':
                if (category[0] != '東証') and (category[0] != '時価総額'):
                    word = '<organization>'
            elif (tag[0] == '助詞') or (word=='、') or \
                    (word=='､') or (word=='・') or (word=='･') or (word == ''):
                continue

            output_words.append(word)
    
    if len(output_words) < 4:
        return None
    else:
        return ':'.join(output_words)

In [13]:
arr1 = df1[:100].apply(Morph, axis=1)
arr2 = df1[:100]['Headline']

In [14]:
for x1,x2 in zip(arr1, arr2):
    print(x1)
    print(x2)
    print()

<person>:<location>:知事:企業:立地:着実:自動車:0:税:撤廃:要請
<日経>◇大村・愛知知事「企業立地、着実に」　自動車２税撤廃要請

日経平均:大幅続伸:始まる:崖:回避:円安:好感:し:震災前:水準:上回る
<ＮＱＮ>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感し震災前水準上回る

日経平均:大幅続伸:始まる:崖:回避:円安:好感
<日経>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感

東証:0部:時価総額:0兆:円:上回る:0年0月:以来
<ＮＱＮ>◇東証１部の時価総額、300兆円上回る　11年３月以来

<company>:一時:0%:超:高:円安:進行:一段:採算:改善:期待
<ＮＱＮ>◇＜東証＞トヨタが一時５％超高　円安進行で一段の採算改善期待

<company>:0年0月:以来:高値:円安:<location>:販売:回復:支援
<ＮＱＮ>◇＜東証＞トヨタが08年10月以来の高値　円安、米販売回復も支援

日経平均:大引け:大幅続伸:震災前:水準:回復:し:0年:0カ月:ぶり:高値
<ＮＱＮ>◇日経平均大引け、大幅続伸　震災前水準回復し１年10カ月ぶり高値

<company>:0年間:工場:新設:せ:ず:既存:工場:効率化
<日経>◇トヨタ、３年間は工場新設せず　既存工場を効率化

<location>:サイバー:教育:研:経営:財務:学べる:ゲーム:販売
<日経>◇日本サイバー教育研、経営・財務学べるゲーム販売

外為:0時:円:0円:台:前半:もみ合い:輸出企業:買い:膨らむ
<ＮＱＮ>◇外為10時　円､88円台前半でもみ合い　輸出企業の買い膨らむ

<company>:イー・アクセス:完全子会社:化:する:株式交換:完了
<日経>◎ソフトバンク、イー・アクセスを完全子会社化する株式交換が完了

<company>:続落:米:ｽﾌﾟﾘﾝﾄ:ｸﾘｱﾜｲｱ:子会社化:差し止め:要請
<ＮＱＮ>◇＜東証＞ｿﾌﾄﾊﾞﾝｸが続落　｢米ｽﾌﾟﾘﾝﾄのｸﾘｱﾜｲｱ子会社化に差し止め要請｣

外為:0時:円:0円:台:前半:もみ合い:輸出入:企業:売買:交錯
<ＮＱＮ>◇外為12時　円､88円台前半でもみ合い　輸出入企業の売買が交錯

株:今週末:SQ:算出:株高:コール:売:建て:買い戻し:最近:0カ月:動向
<ＮＱＮ

In [15]:
re.sub(r"(\*J)", "", "ＩＰＯ：FUJIJPN(1449*J)　初値決定前の気配運用")

'ＩＰＯ：FUJIJPN(1449)\u3000初値決定前の気配運用'

In [16]:
"<QUICK>5%ﾙｰﾙ報告16日 日本ｱｸｱ(1429)―大量(ﾊﾝﾂﾏﾝ･ｲﾝﾍﾞｽﾄﾒﾝﾂ(ﾈｻﾞｰﾗﾝﾄﾞ)ﾋﾞｰｳﾞｨ)".replace("日本ｱｸｱ", "1")

'<QUICK>5%ﾙｰﾙ報告16日 1(1429)―大量(ﾊﾝﾂﾏﾝ･ｲﾝﾍﾞｽﾄﾒﾝﾂ(ﾈｻﾞｰﾗﾝﾄﾞ)ﾋﾞｰｳﾞｨ)'

In [17]:
df4 = df1.copy()
df4['Headline'] = df1.apply(Morph, axis=1)
df4 = df4.dropna(subset=['Headline'])

In [18]:
df4

Headline Company_Name_J  \
date       code                                                                 
2013-01-03 7203  <person>:<location>:知事:企業:立地:着実:自動車:0:税:撤廃:要請         トヨタ自動車   
2013-01-04 7203          日経平均:大幅続伸:始まる:崖:回避:円安:好感:し:震災前:水準:上回る         トヨタ自動車   
           7203                       日経平均:大幅続伸:始まる:崖:回避:円安:好感         トヨタ自動車   
2013-01-05 7203                    東証:0部:時価総額:0兆:円:上回る:0年0月:以来         トヨタ自動車   
           7203          <company>:一時:0%:超:高:円安:進行:一段:採算:改善:期待         トヨタ自動車   
...                                                        ...            ...   
2019-05-18 7203                       <company>:0:第:0:0回:社債:発行         トヨタ自動車   
           7203              <company>:第:0回:第:0回:無担保社債:AA:+:格付         トヨタ自動車   
           7203    <company>:第:0:0回:債:格付け:RI:<organization>:SP         トヨタ自動車   
2019-05-19 7203      <company>:0年:ぶり:社債:発行:額:積み増し:国債:代替:需要:集まる         トヨタ自動車   
           7203        <company>:<location>:スープラ:0年:ぶり:復活:0万:円         トヨタ自動車   

                Company_Relevance  \
date       code                     
2013-01-03 7203                35   
2013-01-04 7203                36   
           7203                36   
2013-01-05 7203                36   
           7203               100   
...                           ...   
2019-05-18 7203               100   
           7203               100   
           7203               100   
2019-05-19 7203               100   
           7203               100   

                                                   Keyword_Article  
date       code                                                     
2013-01-03 7203  愛知県:景気:対策:防災:減災:課題:活性化:自動車取得税:撤廃:国内生産:落ち込む:響く:...  
2013-01-04 7203  日経平均株価:続伸:前年:発生:３月:終値:水準:回復:円安:米国:歳出削減:財政:問題:回...  
           7203  日経平均株価:続伸:前年:上昇:発生:３月:終値:水準:回復:円安:米国:歳出削減:財政:問...  
2013-01-05 7203  時価総額:前年:増加:終値:発生:３月:休場:米国:財政:問題:回避:円安:ハイテク:買い:...  
           7203  大幅:年末年始:続伸:上昇:１月:高値:米国:財政:回避:円売り:外国為替市場:円相場:下落...  
...                                                            ...  
2019-05-18 7203                                        無担保社債:５月:払込  
           7203  新規格付:無担保社債:社債:格付:理由:１月:リリース:発行:名称:５月:償還:新規:担保:...  
           7203                                              無担保社債  
2019-05-19 7203  普通社債:発行条件:社債:発行:当初:想定:積み増し:長期金利:マイナス:国債:運用:苦戦:...  
           7203  エンジン:価格:共同開発:新:モデル:投入:需要:喚起:代表:北米:登場:販売:環境意識:受...  

[7624 rows x 4 columns]

# テキストと株価をくっつける

In [19]:
# 時系列をくっつける
df3 = pd.concat([df4, df2], axis=1, join_axes=[df4.index], levels=[0, 1])
df3 = df3.dropna()
df3 = df3.sort_values(by=['code', 'date'])
df3['Headline'] = \
    df3.groupby(level=[0,1]).apply(lambda x: ':<pad>:'.join(list(x['Headline'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Headline'])]
df3['price'] = \
     df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3 = df3.dropna()

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [20]:
# CSVファイルに保存する
df5 = pd.concat([df3[['Headline', 'price']].rename(
                                   columns={'Headline': 'state', 'price': 'reward'}),
                             df3[['Headline']].shift(-1).rename(
                                   columns={'Headline': 'next_state'})], axis=1).dropna()
df5 = df5[['state', 'next_state', 'reward']]

date_year = df5.index.map(lambda x: x[0].year)

df5[date_year <= args.train_date].to_csv(
    '../../data/news/text_train.tsv',
    header=None,
    index=None,
    sep='\t')

df5[(args.train_date < date_year) & (date_year < args.test_date)].to_csv(
    '../../data/news/text_val.tsv',
    header=None,
    index=None,
    sep='\t')

df5[(args.test_date <= date_year)].to_csv(
    '../../data/news/text_test.tsv',
    header=None,
    index=None,
    sep='\t')

# df5[(args.test_date <= date_year) & (date_year <= 2017)].to_csv(
#     '../../data/news/text_test.tsv',
#     header=None,
#     index=None,
#     sep='\t')

In [16]:
m.parse("-0")

'-0\t名詞,一般,*,*,*,*,マイナスゼロ,マイナスゼロ,マイナスゼロ\nEOS\n'

# 銘柄の名前リストを入れる

In [17]:
# for i, date in enumerate(range(2011, 2020)):
#     tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
# #     tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
#     # tmp = tmp[tmp['Company_Relevance'] == str(100)]
#     tmp = tmp[['Time_Stamp_Original(JST)', 
#                     'Company_Code(TSE)', 
#                     'Headline', 
#                    'Company_Name_J',
#                     'News_Source',
#                     'Company_Relevance', 
#                     'Keyword_Article']]

#     # 欠損除去
#     tmp = tmp[~tmp["Keyword_Article"].isnull()]

#     # タグ除去
#     tmp = tmp[(tmp['News_Source'] == '日経') | 
#                  (tmp['News_Source'] == 'ＮＱＮ') |
#                  (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
#                  (tmp['News_Source'] == 'Ｒ＆Ｉ')]

# #     tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
# #     tmp = tmp.set_index(['date', 'code'], drop=True)
#     tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source'], axis=1)

#     if i == 0:
#         df5 = tmp.copy()
#     else:
#         df5 = pd.concat([df5, tmp])

In [18]:
# df5 = df5[['Company_Code(TSE)', 'Company_Name_J']].dropna()
# df5['Company_Code(TSE)'] = df5['Company_Code(TSE)'].astype(int)
# df5 = df5.sort_values('Company_Code(TSE)')

In [19]:
# df5[~df5.duplicated()].to_csv('../../data/company_name.csv', index=False, header=False)